# base 1d cnn model


In [ ]:
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False
SEED = [0, 1, 2, 3 ,4, 5]

In [ ]:
#ColaboratoryかKaggleNotebookか判別
import sys
import os
from pathlib import Path

if 'google.colab' in sys.modules:  # colab環境
    SEED = [0] # 1つ目のみ
    INPUT = Path('/content/input/')

    # drive mount
    from google.colab import drive
    drive.mount('/content/drive/')

    # working dir に移動
    os.chdir('/content/drive/MyDrive/git/kaggle-lish-moa/working')
    print("cwd:", os.getcwd())
    !pip install iterative-stratification

elif 'kaggle_web_client' in sys.modules:  # kaggle環境
    INPUT = Path('../input/')
    sys.path.append('../input/iterative-stratification/iterative-stratification-master')

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


Mounted at /content/drive/
cwd: /content/drive/MyDrive/git/kaggle-lish-moa/working


In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.listdir('../input/lish-moa')

['train_drug.csv',
 'test_features.csv',
 'train_targets_scored.csv',
 'train_targets_nonscored.csv',
 'train_features.csv',
 'sample_submission.csv']

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
# 相関係数の高い nonscored を抽出
corr = pd.concat([train_targets_scored, train_targets_nonscored],axis=1).corr()
corr[train_targets_nonscored.drop("sig_id",axis=1).columns]
corr_se = corr[:len(train_targets_scored.drop("sig_id",axis=1).columns)][train_targets_nonscored.drop("sig_id",axis=1).columns].abs().max(axis=0).sort_values(ascending=False)
len(corr_se[corr_se>0.3])
non_scored_target_high_corr = list(corr_se[corr_se>0.3].index)
train_targets_nonscored_high_corr = train_targets_nonscored[["sig_id"]+non_scored_target_high_corr]
print(train_targets_nonscored_high_corr)

             sig_id  ...  differentiation_inducer
0      id_000644bb2  ...                        0
1      id_000779bfc  ...                        0
2      id_000a6266a  ...                        0
3      id_0015fd391  ...                        0
4      id_001626bd3  ...                        0
...             ...  ...                      ...
23809  id_fffb1ceed  ...                        0
23810  id_fffb70c0c  ...                        0
23811  id_fffc1c3f4  ...                        0
23812  id_fffcb9e7c  ...                        0
23813  id_ffffdd77b  ...                        0

[23814 rows x 34 columns]


In [ ]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
feat_dic = {}
feat_dic['gene'] = GENES
feat_dic['cell'] = CELLS

def norm_fit(df_1,saveM = True, sc_name = 'zsco'):   
    from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler,RobustScaler,Normalizer,QuantileTransformer,PowerTransformer
    ss_1_dic = {'zsco':StandardScaler(),
                'mima':MinMaxScaler(),
                'maxb':MaxAbsScaler(), 
                'robu':RobustScaler(),
                'norm':Normalizer(), 
                'quan':QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal"),
                'powe':PowerTransformer()}
    ss_1 = ss_1_dic[sc_name]
    df_2 = pd.DataFrame(ss_1.fit_transform(df_1),index = df_1.index,columns = df_1.columns)
    if saveM == False:
        return(df_2)
    else:
        return(df_2,ss_1)
  
def norm_tra(df_1,ss_x):
    df_2 = pd.DataFrame(ss_x.transform(df_1),index = df_1.index,columns = df_1.columns)
    return(df_2)
    
# sample norm 
q2 = train_features[feat_dic['gene']].apply(np.quantile,axis = 1,q = 0.25).copy()
q7 = train_features[feat_dic['gene']].apply(np.quantile,axis = 1,q = 0.75).copy()
qmean = (q2+q7)/2
train_features[feat_dic['gene']] = (train_features[feat_dic['gene']].T - qmean.values).T
q2 = test_features[feat_dic['gene']].apply(np.quantile,axis = 1,q = 0.25).copy()
q7 = test_features[feat_dic['gene']].apply(np.quantile,axis = 1,q = 0.75).copy()
qmean = (q2+q7)/2
test_features[feat_dic['gene']] = (test_features[feat_dic['gene']].T - qmean.values).T

q2 = train_features[feat_dic['cell']].apply(np.quantile,axis = 1,q = 0.25).copy()
q7 = train_features[feat_dic['cell']].apply(np.quantile,axis = 1,q = 0.72).copy()
qmean = (q2+q7)/2
train_features[feat_dic['cell']] = (train_features[feat_dic['cell']].T - qmean.values).T
qmean2 = train_features[feat_dic['cell']].abs().apply(np.quantile,axis = 1,q = 0.75).copy()+4
train_features[feat_dic['cell']] = (train_features[feat_dic['cell']].T / qmean2.values).T.copy()

q2 = test_features[feat_dic['cell']].apply(np.quantile,axis = 1,q = 0.25).copy()
q7 = test_features[feat_dic['cell']].apply(np.quantile,axis = 1,q = 0.72).copy()
qmean = (q2+q7)/2
test_features[feat_dic['cell']] = (test_features[feat_dic['cell']].T - qmean.values).T
qmean2 = test_features[feat_dic['cell']].abs().apply(np.quantile,axis = 1,q = 0.75).copy()+4
test_features[feat_dic['cell']] = (test_features[feat_dic['cell']].T / qmean2.values).T.copy()

#------------ norm --------------
col_num = list(set(feat_dic['gene'] + feat_dic['cell']) )
col_num.sort()
train_features[col_num], ss = norm_fit(train_features[col_num],True,'quan')
test_features[col_num]     = norm_tra(test_features[col_num],ss)

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [ ]:
train_targets_scored.sum()[1:].sort_values()

atp-sensitive_potassium_channel_antagonist      1
erbb2_inhibitor                                 1
diuretic                                        6
autotaxin_inhibitor                             6
protein_phosphatase_inhibitor                   6
                                             ... 
serotonin_receptor_antagonist                 404
dopamine_receptor_antagonist                  424
cyclooxygenase_inhibitor                      435
proteasome_inhibitor                          726
nfkb_inhibitor                                832
Length: 206, dtype: object

In [ ]:
train_features['cp_type'].unique()

array(['trt_cp', 'ctl_vehicle'], dtype=object)

# PCA features + Existing features

In [ ]:
# GENES
n_comp = 50

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [ ]:
#CELLS
n_comp = 15

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

# feature Selection using Variance Encoding

In [ ]:
from sklearn.feature_selection import VarianceThreshold


var_thresh = VarianceThreshold(threshold=0.5)
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features


,sig_id,cp_type,cp_time,cp_dose,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936
0,id_000644bb2,trt_cp,24,D1,1.126588,0.896743,-0.419975,-0.987724,-0.261362,-1.019833,-1.357432,-0.036391,0.683932,-0.312484,1.544427,0.169884,0.629200,-0.560967,0.284361,-1.063809,-1.140764,0.868345,0.382571,-0.515578,-0.744083,-1.308057,-1.685607,1.232221,0.551543,0.398992,0.238473,0.166220,-0.535662,0.779434,0.414264,-1.121055,-0.059489,-0.445588,-0.204124,0.266637,...,1.874684,0.728876,-0.682201,-0.867972,-0.727414,0.896742,-0.160439,0.201803,-1.701895,0.038630,0.217904,-1.560113,-0.251815,2.059157,0.177284,0.753183,0.172003,0.254249,1.103743,-2.423089,0.282657,-0.056549,-0.550481,0.470953,-0.819566,0.155108,1.129632,1.769502,0.061783,-1.103373,-0.687035,2.363661,-0.119732,1.058464,-0.396841,-0.482863,1.082089,-0.608718,-0.950615,-0.598907
1,id_000779bfc,trt_cp,72,D1,0.123960,0.686144,0.281471,0.089220,1.197490,0.695488,0.325732,0.558904,-0.528336,0.846367,-1.255438,-0.563654,-0.200083,0.549301,0.162757,0.402200,0.406731,0.410560,-0.831429,0.511447,1.256254,-0.124842,-0.387620,-0.415918,0.400921,-0.579409,0.651503,0.234805,-0.737432,-0.179216,-0.106861,-0.537304,1.652468,-0.360877,0.367221,-0.269058,...,-2.507209,0.812504,-0.494634,0.511296,0.945359,2.548873,0.652772,-1.680784,-1.395383,2.575009,-0.298309,1.074672,1.706043,-0.366847,-1.130237,1.450744,-0.765327,-0.052048,-0.485376,0.853620,-1.366777,0.516627,0.262665,-0.224431,-1.143572,-0.334881,-1.205319,-0.054506,-0.800481,-0.276697,0.462977,-0.174409,-0.245746,-2.197200,0.679922,0.844831,0.382629,0.042944,0.224284,-0.149290
2,id_000a6266a,trt_cp,48,D1,0.791802,0.963103,1.446197,-0.104188,0.016850,1.506677,0.259336,0.388398,0.018690,1.263162,-0.631766,-0.744935,-0.104532,-2.272928,0.915987,-0.516485,0.488828,-0.381854,-0.264876,-0.014108,-0.085669,-0.977670,-1.943257,0.552849,0.644842,0.732406,-1.375768,2.391415,-0.062331,1.604275,-1.456559,0.835065,0.757337,0.233871,0.026929,-1.628269,...,-0.397246,-1.556993,-1.364556,-1.276963,-0.575073,1.139321,-2.757371,-0.148768,1.124297,-1.284978,2.288263,3.239749,-3.147330,1.647471,2.900226,0.035748,1.511792,-0.538020,-0.514330,1.722944,-0.042691,-2.499278,-0.361552,-0.065624,-3.439062,0.360763,0.236295,-0.459681,0.782969,-0.706135,0.228319,0.043046,0.128723,0.095275,-0.093033,-1.067634,-0.158550,-0.425903,1.510575,-0.614885
3,id_0015fd391,trt_cp,48,D1,-0.678723,-0.225107,-0.390217,0.817606,2.347629,-0.829998,-2.240530,0.358999,-0.144604,-1.366325,-0.983661,-0.448022,-1.119056,-0.759329,-1.758028,1.454682,-0.186508,-1.017801,0.333821,-2.112178,2.098888,-1.292435,-1.057987,-1.011599,-1.391260,0.050921,-1.030224,0.311702,-0.397560,-1.224687,-0.954690,0.193184,-2.071169,1.082679,0.848135,-2.098790,...,-0.403723,0.799991,-0.846906,-1.825898,0.275374,3.215788,2.466925,2.896389,-1.712297,1.412401,-1.530897,1.101305,-5.110287,-4.281156,1.345197,-1.717697,0.219488,1.444570,2.686697,-0.720024,1.434588,1.635843,0.502135,1.529322,-0.407975,5.915576,7.043598,-1.278447,-3.642887,-0.170505,1.328007,-0.341669,-0.700178,-0.568794,0.617534,-1.144837,2.252996,-0.307845,2.358449,-0.733371
4,id_001626bd3,trt_cp,72,D2,-0.455668,-0.480522,0.967960,0.978515,1.455881,-0.875480,-0.378600,-0.213123,-1.067782,0.846761,-0.349202,-0.708051,-1.257696,1.176129,0.319188,0.333307,-0.030044,-1.567980,-0.863951,-0.708170,-0.464143,-0.291901,1.055318,0.481658,1.671455,1.008205,1.084597,-0.562678,-1.550207,0.376295,-0.157058,0.842966,1.108839,-0.143759,1.506831,-1.012107,...,-2.842882,0.590315,0.866107,0.838733,0.742959,-0.048764,1.566977,-0.588171,-1.045144,3.052931,-1.138468,-1.484278,0.894380,-2.876091,-0.384195,-0.206759,-2.312379,-0.468270,1.313270,1.193817,0.719543,-0.504175,0.625338,0.591882,0.430694,-0.147733,0.656648,-0.200896,1.242065,-0.432718,-0.425102,0.018450,0.03

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored_high_corr, on='sig_id')

train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

#----
# augmentation for less target
skew_target_cols = train_targets_scored.drop('sig_id', axis=1).sum()[lambda x:x<10].index.values
print(skew_target_cols)
augmented_num = 0
for col in skew_target_cols:
    tmp_df = train[train[col]==1].copy()
for _ in range(4):
    train = pd.concat((train, tmp_df))
    augmented_num += len(tmp_df.index) * 4
print(train.shape)
train = train.reset_index(drop=True)
#---
target = train[train_targets_scored.columns]

['aldehyde_dehydrogenase_inhibitor' 'antiarrhythmic'
 'atm_kinase_inhibitor' 'atp-sensitive_potassium_channel_antagonist'
 'autotaxin_inhibitor' 'bacterial_membrane_integrity_inhibitor'
 'calcineurin_inhibitor' 'coagulation_factor_inhibitor' 'diuretic'
 'elastase_inhibitor' 'erbb2_inhibitor' 'laxative' 'leukotriene_inhibitor'
 'lxr_agonist' 'nicotinic_receptor_agonist'
 'norepinephrine_reuptake_inhibitor' 'protein_phosphatase_inhibitor'
 'retinoid_receptor_antagonist' 'steroid' 'tlr_antagonist'
 'tropomyosin_receptor_kinase_inhibitor'
 'ubiquitin_specific_protease_inhibitor']
(21972, 1180)


In [ ]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [ ]:
train

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,abc_transporter_expression_enhancer,dna_methyltransferase_inhibitor,ror_inverse_agonist,nfkb_activator,sars_coronavirus_3c-like_protease_inhibitor,glucocorticoid_receptor_antagonist,macrophage_migration_inhibiting_factor_inhibitor,heme_oxygenase_activators,glutathione_reductase_(nadph)_activators,diacylglycerol_o_acyltransferase_inhibitor,keap1_ligand,steryl_sulfatase_inhibitor,hiv_protease_inhibitor,quorum_sensing_signaling_modulator,camp_stimulant,macrophage_inhibitor,abl_inhibitor,membrane_permeability_inhibitor,ephrin_inhibitor,gaba_gated_chloride_channel_blocker,omega_3_fatty_acid_stimulant,niemann-pick_c1-like_1_protein_antagonist,gap_junction_modulator,dna_dependent_protein_kinase_inhibitor,reducing_agent,big1_inhibitor,tyrosine_phosphatase_inhibitor,hgf_receptor_inhibitor,caspase_inhibitor,selective_estrogen_receptor_modulator_(serm),imidazoline_ligand,sphingosine_1_phosphate_receptor_agonist,differentiation_inducer
0,id_000644bb2,24,D1,1.126588,0.896743,-0.419975,-0.987724,-0.261362,-1.019833,-1.357432,-0.036391,0.683932,-0.312484,1.544427,0.169884,0.629200,-0.560967,0.284361,-1.063809,-1.140764,0.868345,0.382571,-0.515578,-0.744083,-1.308057,-1.685607,1.232221,0.551543,0.398992,0.238473,0.166220,-0.535662,0.779434,0.414264,-1.121055,-0.059489,-0.445588,-0.204124,0.266637,0.381819,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,72,D1,0.123960,0.686144,0.281471,0.089220,1.197490,0.695488,0.325732,0.558904,-0.528336,0.846367,-1.255438,-0.563654,-0.200083,0.549301,0.162757,0.402200,0.406731,0.410560,-0.831429,0.511447,1.256254,-0.124842,-0.387620,-0.415918,0.400921,-0.579409,0.651503,0.234805,-0.737432,-0.179216,-0.106861,-0.537304,1.652468,-0.360877,0.367221,-0.269058,0.210515,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,48,D1,0.791802,0.963103,1.446197,-0.104188,0.016850,1.506677,0.259336,0.388398,0.018690,1.263162,-0.631766,-0.744935,-0.104532,-2.272928,0.915987,-0.516485,0.488828,-0.381854,-0.264876,-0.014108,-0.085669,-0.977670,-1.943257,0.552849,0.644842,0.732406,-1.375768,2.391415,-0.062331,1.604275,-1.456559,0.835065,0.757337,0.233871,0.026929,-1.628269,0.171128,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,48,D1,-0.678723,-0.225107,-0.390217,0.817606,2.347629,-0.829998,-2.240530,0.358999,-0.144604,-1.366325,-0.983661,-0.448022,-1.119056,-0.759329,-1.758028,1.454682,-0.186508,-1.017801,0.333821,-2.112178,2.098888,-1.292435,-1.057987,-1.011599,-1.391260,0.050921,-1.030224,0.311702,-0.397560,-1.224687,-0.954690,0.193184,-2.071169,1.082679,0.848135,-2.098790,-1.399664,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,72,D2,-0.455668,-0.480522,0.967960,0.978515,1.455881,-0.875480,-0.378600,-0.213123,-1.067782,0.846761,-0.349202,-0.708051,-1.257696,1.176129,0.319188,0.333307,-0.030044,-1.567980,-0.863951,-0.708170,-0.464143,-0.291901,1.055318,0.481658,1.671455,1.008205,1.084597,-0.562678,-1.550207,0.376295,-0.157058,0.842966,1.108839,-0.143759,1.506831,-1.012107,-1.331879,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21967,id_2c1f61f65,72,D2,-0.052487,-0.090507,0.634989,0.970022,0.622500,0.058178,0.543512,-0.594642,-0.360361,-1.218731,0.958293,0.761964,-0.271883,-0.683985,-0.375000,0

In [ ]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

# CV folds

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
21967   NaN
21968   NaN
21969   NaN
21970   NaN
21971   NaN
Name: kfold, Length: 21972, dtype: float64

In [ ]:
train_drug = pd.read_csv('../input/lish-moa/train_drug.csv')

folds = train.copy()
def make_fold():
    folds = train.copy()
    """
    # LOCATE DRUGS
    vc = train_drug.drug_id.value_counts()
    vc1 = vc.loc[vc <= 19].index
    vc2 = vc.loc[vc > 19].index


    # kfold - leave drug out target2 = target.copy()
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits = 5) # , shuffle = True, random_state = seed
    tmp = target2.groupby('drug_id')[target_cols].mean().loc[vc1]
    tmp_idx = tmp.index.tolist()
    tmp_idx.sort()
    tmp_idx2 = random.sample(tmp_idx,len(tmp_idx))
    tmp = tmp.loc[tmp_idx2]
    for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)

    skf = MultilabelStratifiedKFold(n_splits = 5) # , shuffle = True, random_state = seed
    tmp = target2.loc[target2.drug_id.isin(vc2)].reset_index(drop = True)
    tmp_idx = tmp.index.tolist()
    tmp_idx.sort()
    tmp_idx2 = random.sample(tmp_idx,len(tmp_idx))
    tmp = tmp.loc[tmp_idx2]
    for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)

    target2['kfold'] = target2.drug_id.map(dct1)
    target2.loc[target2.kfold.isna(),'kfold'] = target2.loc[target2.kfold.isna(),'sig_id'].map(dct2)
    target2.kfold = target2.kfold.astype(int)
    folds['kfold'] = target2['kfold'].copy()

    """
    folds = train.copy()

    mskf = MultilabelStratifiedKFold(n_splits=5)

    for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
        folds.loc[v_idx, 'kfold'] = int(f)

    folds['kfold'] = folds['kfold'].astype(int)
    folds

    return folds


In [ ]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

(21972, 1179)
(21972, 1180)
(3624, 940)
(21972, 207)
(3982, 207)


# Dataset Classes

In [ ]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   
    

# Model

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        cha_1 = 256
        cha_2 = 512
        cha_3 = 512

        cha_1_reshape = int(hidden_size/cha_1)
        cha_po_1 = int(hidden_size/cha_1/2)
        cha_po_2 = int(hidden_size/cha_1/2/2) * cha_3

        self.cha_1 = cha_1
        self.cha_2 = cha_2
        self.cha_3 = cha_3
        self.cha_1_reshape = cha_1_reshape
        self.cha_po_1 = cha_po_1
        self.cha_po_2 = cha_po_2

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

        self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
        self.dropout_c1 = nn.Dropout(0.1)
        self.conv1 = nn.utils.weight_norm(nn.Conv1d(cha_1,cha_2, kernel_size = 5, stride = 1, padding=2,  bias=False),dim=None)

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

        self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2 = nn.Dropout(0.1)
        self.conv2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_1 = nn.Dropout(0.3)
        self.conv2_1 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_2, kernel_size = 3, stride = 1, padding=1, bias=True),dim=None)

        self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_2 = nn.Dropout(0.2)
        self.conv2_2 = nn.utils.weight_norm(nn.Conv1d(cha_2,cha_3, kernel_size = 5, stride = 1, padding=2, bias=True),dim=None)

        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x), alpha=0.06)

        x = x.reshape(x.shape[0],self.cha_1,self.cha_1_reshape)

        x = self.batch_norm_c1(x)
        x = self.dropout_c1(x)
        x = F.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c2_1(x)
        x = self.dropout_c2_1(x)
        x = F.relu(self.conv2_1(x))

        x = self.batch_norm_c2_2(x)
        x = self.dropout_c2_2(x)
        x = F.relu(self.conv2_2(x))
        x =  x * x_s

        x = self.max_po_c2(x)

        x = self.flt(x)

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)

        return x

from torch.nn.modules.loss import _WeightedLoss
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction
    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets
    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight,
                                                      pos_weight = pos_weight)
        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()
        return loss

# Preprocessing steps

In [ ]:
def process_data(data):
    
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    
    return data

In [ ]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in non_scored_target_high_corr]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)

942

In [ ]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=4096


# Single fold training

In [ ]:
print(process_data(folds).shape)
print(process_data(test).shape)


(21972, 1183)
(3624, 943)


In [ ]:


def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)


    #----
    # pretrain non_scored_target_high_corr
    print("pretrain ")
    print(" start")
    loss_fr = nn.BCEWithLogitsLoss()
    loss_va = nn.BCEWithLogitsLoss()    
    model = Model(
        num_features=num_features,
        num_targets=len(non_scored_target_high_corr),
        hidden_size=hidden_size,
    )
    model.to(DEVICE)

    x_train, y_train  = train_df[feature_cols].values, train_df[non_scored_target_high_corr].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[non_scored_target_high_corr].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE*0.1, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5,max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, eta_min=0, last_epoch=-1, verbose=False)

    for epoch in range(1):
        train_loss = train_fn(model, optimizer,scheduler, loss_fr, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, loss_va, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch},train_loss: {train_loss}, valid_loss: {valid_loss}")

    model.dense3 = nn.utils.weight_norm(nn.Linear(model.cha_po_2, num_targets))
    model.to(DEVICE)
    #----


    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3,   max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, eta_min=0, last_epoch=-1, verbose=False)

    loss_tr = SmoothBCEwLogits(smoothing = 0.001)
    loss_va = nn.BCEWithLogitsLoss()    

    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer, scheduler, loss_fr, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_va, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions


In [ ]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [ ]:
# Averaging on multiple SEEDS


oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    folds = make_fold()
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions


pretrain 
 start
FOLD: 0, EPOCH: 0,train_loss: 0.736991957909819, valid_loss: 0.7079861981528146
FOLD: 0, EPOCH: 0, train_loss: 0.46732067677151895
FOLD: 0, EPOCH: 0, valid_loss: 0.0238169651478529
FOLD: 0, EPOCH: 1, train_loss: 0.020228623007626637
FOLD: 0, EPOCH: 1, valid_loss: 0.018297283085329193
FOLD: 0, EPOCH: 2, train_loss: 0.018243167799074148
FOLD: 0, EPOCH: 2, valid_loss: 0.018094169641179696
FOLD: 0, EPOCH: 3, train_loss: 0.017344260196862877
FOLD: 0, EPOCH: 3, valid_loss: 0.017836965726954597
FOLD: 0, EPOCH: 4, train_loss: 0.01687898152116416
FOLD: 0, EPOCH: 4, valid_loss: 0.01765461739684854
FOLD: 0, EPOCH: 5, train_loss: 0.016783716494514458
FOLD: 0, EPOCH: 5, valid_loss: 0.019144801050424577
FOLD: 0, EPOCH: 6, train_loss: 0.01676632184535265
FOLD: 0, EPOCH: 6, valid_loss: 0.017755941674113273
FOLD: 0, EPOCH: 7, train_loss: 0.016853208464664825
FOLD: 0, EPOCH: 7, valid_loss: 0.017584330295877797
FOLD: 0, EPOCH: 8, train_loss: 0.01672674286975593
FOLD: 0, EPOCH: 8, valid_l

In [52]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train.iloc[:-augmented_num,:][['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
print(len(train_targets_scored))
print(len(valid_results))

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)
    

23814
23814
CV log_loss:  0.016161400758170064


In [53]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)

In [54]:
sub.shape

(3982, 207)